In [2]:
%reload_ext autoreload
%autoreload 2

import sys
from tqdm import tqdm
import h5py
import numpy as np

sys.path.append("..")
import axionmirror.units_constants as uc

from config import pc_dict

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import colormaps as cms
mpl.rc_file('../matplotlibrc')

In [62]:
nu_ref = 400. # [MHz]
t_ref = pc_dict['CHIME'].telescope
sens_ref = t_ref.sens_estimate([nu_ref])[0]
merit_ref = t_ref.merit(nu_ref)

print(f'At {nu_ref:.0f} MHz')
print('name     Aeff[m^2]  FOV[deg^2] n_pixel tobs/day[min] sqrt(npol*Aeff*FOV) sens(rel.)')
pc_names = ['CHIME', 'HERA', 'CHORD', 'HIRAX256', 'HIRAX1024', 'BURSTT256', 'BURSTT2048']

for pc_name in pc_names:
    pc = pc_dict[pc_name]
    t = pc.telescope
    
    print(f'{t.name:12}', end='')
    A = t.Aeff_zenith(nu_ref)/10000 # [m^2]
    print(f'{A:6.0f}', end='')
    FOV = t.instantaneous_fov(nu_ref)*(180/np.pi)**2 # [deg^2]
    print(f'{FOV:12.0f}', end='')
    pixel_size = (uc.c0 / nu_ref) ** 2 / t.Aeff_zenith(nu_ref) # [rad^2]
    n_pixel = t.survey_area(nu_ref) / pixel_size
    print(f'{n_pixel:8.0f}', end='')
    print(f'{t.t_obs(nu_ref, t.dec)/t.t_obs_days/60:14.1f}', end='')
    print(f'{t.merit(nu_ref) / merit_ref:15.3f}', end='')
    print(f'{(t.sens_estimate([nu_ref])[0] / sens_ref) ** (-2):11.3f}')

At 400 MHz
name     Aeff[m^2]  FOV[deg^2] n_pixel tobs/day[min] sqrt(npol*Aeff*FOV) sens(rel.)
CHIME         3120         515   41379          26.3          1.000      1.000
HERA         29548          16   19637          18.4          0.537      0.323
CHORD         7173         102   52317          10.5          0.676      0.815
HIRAX256      3629         102   34902           7.9          0.481      0.446
HIRAX1024    14515         102  139607           7.9          0.962      0.892
BURSTT256      131        3600    1343         262.1          0.383      0.462
BURSTT2048    1037        3600   10624         262.1          1.078      1.299


In [7]:
nu_ref = 400. # [MHz]
t_ref = pc_dict['CHIME'].telescope
sens_ref = t_ref.sens_estimate([nu_ref])[0]
merit_ref = t_ref.merit(nu_ref)

print(f'At {nu_ref:.0f} MHz')
print('name     Central freq. Aeff[m^2]  FOV[deg^2] sqrt(npol*Aeff*FOV) sens(rel.)')
pc_names = ['CHIME', 'HERA', 'CHORD', 'HIRAX256', 'HIRAX1024', 'BURSTT256', 'BURSTT2048']

for pc_name in pc_names:

    pc = pc_dict[pc_name]

    t = pc.telescope
    print(f'{t.name:12}', end='')

    nu = (t.nu_max + t.nu_min) / 2
    print(f'{nu:8.0f}', end='')

    A = t.Aeff_zenith(nu)/10000 # [m^2]
    print(f'{A:6.0f}', end='')

    FOV = t.instantaneous_fov(nu)*(180/np.pi)**2 # [deg^2]
    print(f'{FOV:12.0f}', end='')
    
    merit = t.merit(nu)
    if pc_name == 'CHIME':
        merit_ref = merit
    print(f'{merit / merit_ref:15.3f}', end='')

    with h5py.File(f"../outputs/plot_data/{pc.name}/total-base.h5", 'r') as hf:
        nu_arr = hf['nu'][:]
        gagg_arr = hf['gagg'][:]

    gagg_med = np.median(gagg_arr, axis=0)
    sens_est = np.interp(nu, nu_arr, gagg_med)
    if pc_name == 'CHIME':
        sens_est_ref = sens_est
    print(f'{(sens_est / sens_est_ref)**(-2):11.3f}')

At 400 MHz
name     Central freq. Aeff[m^2]  FOV[deg^2] sqrt(npol*Aeff*FOV) sens(rel.)
CHIME            600  3120         344          1.000      1.000
HERA             150 29548         112          1.753      2.329
CHORD            900  7173          20          0.368      0.355
HIRAX256         600  3629          46          0.393      0.598
HIRAX1024        600 14515          46          0.785      1.571
BURSTT256        550    69        3600          0.341      0.206
BURSTT2048       550   549        3600          0.960      1.050
